In [1]:
from pulp import *
import pandas as pd

# Actividad 1

## Datos

##### Potencias mínimas

In [2]:
Pc_min = 150
Pg_min = 75
Po_min = 0

##### Potencias máximas

In [3]:
Pc_max = 350
Pg_max = 150
Po_max = 100

##### Costos fijos

In [4]:
cfc = 1
cfg = 0.5
cfo = 0.1

##### Costos variables

In [5]:
cvc = 40
cvg = 75
cvo = 110

##### Demanda

In [6]:
D = 500

## Problema de optmización primal:
### Minimización de costos

##### Inicialización del problema

In [7]:
prob = LpProblem("Primal", LpMinimize)

##### Definición de variables

In [8]:
Pc = LpVariable("Pc")
Pg = LpVariable("Pg")
Po = LpVariable("Po")

##### Función objetivo

In [9]:
prob += (cfc + cvc * Pc) + (cfg + cvg * Pg) + (cfo + cvo * Po)

##### Restricciones

In [10]:
prob += D - (Pc + Pg + Po) == 0 # lambda
prob += Pc_min - Pc <= 0 # mc_min
prob += Pc - Pc_max <= 0 # mc_max
prob += Pg_min - Pg <= 0 # mg_min
prob += Pg - Pg_max <= 0 # mg_max
prob += Po_min - Po <= 0 # mo_min
prob += Po - Po_max <= 0 # mo_max

##### Resolución

In [11]:
prob.solve()

1

##### Estado de resolución del problema

In [12]:
print(LpStatus[prob.status])

Optimal


##### Potencias en el punto óptimo

In [13]:
for potencia in prob.variables():
    print(potencia.name , "=", potencia.varValue, "[MW]")

Pc = 350.0 [MW]
Pg = 150.0 [MW]
Po = 0.0 [MW]


##### Costo total del sistema

In [14]:
print("Costo total = ", value(prob.objective), "[USD]")

Costo total =  25251.6 [USD]


##### Multiplicadores de Lagrange

In [15]:
for restriccion in prob.constraints.values():
    print("Variable dual para la restricción", restriccion, "->", restriccion.pi)

Variable dual para la restricción -Pc - Pg - Po = -500 -> -110.0
Variable dual para la restricción -Pc <= -150 -> 0.0
Variable dual para la restricción Pc <= 350 -> -70.0
Variable dual para la restricción -Pg <= -75 -> 0.0
Variable dual para la restricción Pg <= 150 -> -35.0
Variable dual para la restricción -Po <= 0 -> 0.0
Variable dual para la restricción Po <= 100 -> 0.0


##### Precios sombra

In [16]:
for P in prob.variables():
    print("Precio sombra de", P, "=", P.dj)

Precio sombra de Pc = 0.0
Precio sombra de Pg = 0.0
Precio sombra de Po = 0.0


## Problema de optmización dual:
### Maximización de los beneficios

##### Inicialización del problema

In [17]:
prob = LpProblem("Dual", LpMaximize)

##### Definición de variables

In [18]:
l = LpVariable("l")
mc_min = LpVariable("mc_min")
mc_max = LpVariable("mc_max")
mg_min = LpVariable("mg_min")
mg_max = LpVariable("mg_max")
mo_min = LpVariable("mo_min")
mo_max = LpVariable("mo_max")

##### Función objetivo

In [19]:
prob += D * l + (Pc_min * mc_min + Pg_min * mg_min + Po_min * mo_min) - (Pc_max * mc_max + Pg_max * mg_max + Po_max * mo_max)

##### Restricciones

In [20]:
prob += (l + mc_min - mc_max) - cvc <= 0 # Pc
prob += (l + mg_min - mg_max) - cvg <= 0 # Pg
prob += (l + mo_min - mo_max) - cvo <= 0 # Po
prob += -mc_min <= 0
prob += -mc_max <= 0
prob += -mg_min <= 0
prob += -mg_max <= 0
prob += -mo_min <= 0
prob += -mo_max <= 0

##### Resolución

In [21]:
prob.solve()

1

##### Estado de resolución del problema

In [22]:
print(LpStatus[prob.status])

Optimal


##### Multiplicadores de Lagrange en el punto óptimo

In [23]:
for m in prob.variables():
    print(m.name , "=", m.varValue, "[USD/MW]")

l = 75.0 [USD/MW]
mc_max = 35.0 [USD/MW]
mc_min = -0.0 [USD/MW]
mg_max = -0.0 [USD/MW]
mg_min = -0.0 [USD/MW]
mo_max = -0.0 [USD/MW]
mo_min = -0.0 [USD/MW]


##### Beneficio total del sistema

In [24]:
print("Beneficio total = ", value(prob.objective), "[USD]")

Beneficio total =  25250.0 [USD]


##### Multiplicadores de Lagrange

In [25]:
for restriccion in prob.constraints.values():
    print("Variable dual para la restricción", restriccion, "->", restriccion.pi)

Variable dual para la restricción l - mc_max + mc_min <= 40 -> 350.0
Variable dual para la restricción l - mg_max + mg_min <= 75 -> 150.0
Variable dual para la restricción l - mo_max + mo_min <= 110 -> -0.0
Variable dual para la restricción -mc_min <= 0 -> 200.0
Variable dual para la restricción -mc_max <= 0 -> -0.0
Variable dual para la restricción -mg_min <= 0 -> 75.0
Variable dual para la restricción -mg_max <= 0 -> 0.0
Variable dual para la restricción -mo_min <= 0 -> -0.0
Variable dual para la restricción -mo_max <= 0 -> 100.0


##### Precios sombra
###### ¿Qué son los precios sombra de las variables duales (Multiplicadores de Lagrange) en el problema dual?

In [26]:
for m in prob.variables():
    print("Precio sombra de", m, "=", m.dj)

Precio sombra de l = 0.0
Precio sombra de mc_max = 0.0
Precio sombra de mc_min = 0.0
Precio sombra de mg_max = 0.0
Precio sombra de mg_min = 0.0
Precio sombra de mo_max = 0.0
Precio sombra de mo_min = 0.0


# Actividad 2

In [34]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df = pd.read_excel("Datos_gen.xlsx", sheet_name="Perfil_horario")
print(df.head())